In [115]:
import tensorflow.compat.v1 as tf
from tensorflow.examples.tutorials.mnist import input_data
import warnings
import numpy as np
from PIL import Image
from sklearn.linear_model import LogisticRegression
from tensorflow.python.keras.layers import Dense, Dropout, Activation
tf.compat.v1.disable_eager_execution()

__Test accuracy rate by data points from 6601 to 10000__

In [116]:
warnings.filterwarnings("ignore")
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)  # y labels are oh-encoded
n_train = mnist.train.num_examples  # 55,000
n_validation = mnist.validation.num_examples  # 5000
n_test = mnist.test.num_examples  # 10,000

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [117]:
n_input = 784  # input layer (28x28 pixels)
n_hidden1 = 512  # 1st hidden layer
n_hidden2 = 256  # 2nd hidden layer
n_hidden3 = 128  # 3rd hidden layer
n_output = 10  # output layer (0-9 digits)
learning_rate = 1e-4
n_iterations = 1000
batch_size = 128
dropout = 0.5
cut_start = 6601

In [118]:
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_output])
keep_prob = tf.placeholder(tf.float32)

In [119]:
weights = {
    'w1': tf.Variable(tf.truncated_normal([n_input, n_hidden1], stddev=0.1)),
    'w2': tf.Variable(tf.truncated_normal([n_hidden1, n_hidden2], stddev=0.1)),
    'w3': tf.Variable(tf.truncated_normal([n_hidden2, n_hidden3], stddev=0.1)),
    'out': tf.Variable(tf.truncated_normal([n_hidden3, n_output], stddev=0.1)),
}

biases = {
    'b1': tf.Variable(tf.constant(0.1, shape=[n_hidden1])),
    'b2': tf.Variable(tf.constant(0.1, shape=[n_hidden2])),
    'b3': tf.Variable(tf.constant(0.1, shape=[n_hidden3])),
    'out': tf.Variable(tf.constant(0.1, shape=[n_output]))
}

layer_1 = tf.add(tf.matmul(X, weights['w1']), biases['b1'])
layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
layer_3 = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
layer_drop = tf.nn.dropout(layer_3, keep_prob)
output_layer = tf.matmul(layer_drop, weights['out']) + biases['out']

In [120]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
        labels=Y, logits=output_layer
        ))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [121]:
correct_pred = tf.equal(tf.argmax(output_layer, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [122]:
accuracy_drop_list = []
iteration = 1
while iteration <= 5:
    for i in range(n_iterations):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        sess.run(train_step, feed_dict={
            X: batch_x, Y: batch_y, keep_prob: dropout
            })

        # print loss and accuracy (per minibatch)
        if i % 100 == 0:
            minibatch_loss, minibatch_accuracy = sess.run(
                [cross_entropy, accuracy],
                feed_dict={X: batch_x, Y: batch_y, keep_prob: 1.0}
                )
#             print(
#                 "Iteration",
#                 str(i),
#                 "\t| Loss =",
#                 str(minibatch_loss),
#                 "\t| Accuracy =",
#                 str(minibatch_accuracy)
#                 )
    X_test = mnist.test.images[cut_start:]
    Y_test = mnist.test.labels[cut_start:]
    test_accuracy = sess.run(accuracy, feed_dict={X: X_test, Y: Y_test, keep_prob: 1.0})
    accuracy_drop_list.append(test_accuracy)
    iteration += 1

In [123]:
np.mean(accuracy_drop_list)

0.94463074

__Re-do the entire exercise but without using the dropouts from the final layer. Calculate accuracy of your new model and compare against your previously achieved accuracy. What is your conclusion? Please also explain the intuition behind dropouts.__

In [124]:
layer_1 = tf.add(tf.matmul(X, weights['w1']), biases['b1'])
layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
layer_3 = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
output_layer = tf.matmul(layer_3, weights['out']) + biases['out']

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
        labels=Y, logits=output_layer
        ))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_pred = tf.equal(tf.argmax(output_layer, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [125]:
accuracy_list = []
iteration = 1
while iteration <= 5:
    for i in range(n_iterations):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        sess.run(train_step, feed_dict={
            X: batch_x, Y: batch_y
            })

        # print loss and accuracy (per minibatch)
        if i % 100 == 0:
            minibatch_loss, minibatch_accuracy = sess.run(
                [cross_entropy, accuracy],
                feed_dict={X: batch_x, Y: batch_y}
                )
#             print(
#                 "Iteration",
#                 str(i),
#                 "\t| Loss =",
#                 str(minibatch_loss),
#                 "\t| Accuracy =",
#                 str(minibatch_accuracy)
#                 )
    X_test = mnist.test.images[cut_start:]
    Y_test = mnist.test.labels[cut_start:]
    test_accuracy = sess.run(accuracy, feed_dict={X: X_test, Y: Y_test})
    accuracy_list.append(test_accuracy)
    iteration += 1

In [126]:
np.mean(accuracy_list)

0.9449838

__What other conventional algorithm(s) could you have used for this task? What was the advantage of using a neural network for this exercise, instead of any other potential conventional algorithms?__

Since y variable is categorical range from 0 to 9, we can use multinomial logistic regression to identify the number. However, as 28 * 28 pixels, x variable is highly dimensional, with which MNL is hard to handle. The advantage of Neural network to be applied in this exercise is that three layers are utilized to figure out a better combination of weights that in a way reduces dimension.

__Use a multinomial logistic regression model to achieve the same handwriting identification task and compare the out-of-sample accuracy against the results of the neural network.__

In [46]:
X_train = mnist.train.images
Y_train_array = mnist.train.labels
Y_train = np.array([list(array).index(1.) for array in Y_train_array])
clf = LogisticRegression(random_state=0, multi_class='multinomial', solver='newton-cg')
model = clf.fit(X_train, Y_train)

In [20]:
Y_test_cat = np.array([list(array).index(1.) for array in Y_test])
model.score(X_test, Y_test_cat)

0.923310093967869